In [30]:
import numpy as np
from utils.utils import split_by_passes
from utils.data import load_data

In [231]:
def PQ(y_true, y_pred):
    #TODO: make faster
    
    y_true_sp = split_by_passes(y_true)
    y_true_rngs = np.array([set(range(y[0], y[-1] + 1)) for y in y_true_sp])

    y_pred_sp = split_by_passes(y_pred)
    y_pred_rngs = np.array([set(range(y[0], y[-1] + 1)) for y in y_pred_sp])

    undetected = 0
    for true_pass in y_true_sp:
        if len(split_by_passes(y_pred[true_pass[0]:true_pass[-1]+1])) == 0:
            undetected += 1
    
    #c
    gapped_num = 0
    not_gapped = set(range(len(y_true_sp)))
    for num, true_pass in enumerate(y_true_sp):
        if len(split_by_passes(y_pred[true_pass[0]:true_pass[-1]+1])) > 1:
            gapped_num += 1
            not_gapped.remove(num)

    not_gapped = list(np.array(y_true_rngs)[list(not_gapped)])

    #glued
    glued_num = 0
    not_glued = set(range(len(y_pred_sp)))
    for num, pred_pass in enumerate(y_pred_sp):
        if len(split_by_passes(y_true[pred_pass[0]:pred_pass[-1]+1])) > 1:
            glued_num += 1
            not_glued.remove(num)
            
    not_glued = list(np.array(y_pred_rngs)[list(not_glued)])
    
    #ghost is correct
    ghosts = 0
    for pred_pass in y_pred_sp:
        if len(split_by_passes(y_true[pred_pass[0]:pred_pass[-1]+1])) == 0:
            ghosts += 1
    
    #right detected passes
    #R = np.sum(np.sum(joints, axis=1) == 1)
    R = 0
    
    for true_pass in not_gapped:
        for pred_pass in not_glued:
            if not true_pass.isdisjoint(pred_pass):
                R += 1
                
    
    
    
    #PQ
    return float(R)/(R + undet + gapped_num + glued_num + ghosts)#, R, undet, gapped_num, glued_num, ghosts

In [232]:
data = load_data()

In [233]:
X, y = data.drop(['ref_pass'], axis=1), data['ref_pass']

In [234]:
y_true, y_pred = y.values, data['pass.1'].values

In [235]:
PQ(y_true, y_pred)

0.8407751734974467

In [228]:
6421 + 244 + 323

6988

In [230]:
len(y_true_sp)

7480

In [247]:
import theano.tensor as T
from theano import function

In [248]:
y_true = T.dvector('y_true')

In [249]:
y_pred = T.dvector('y_pred')

In [253]:
result = T.mean((y_true - y_pred) ** 2)

In [ ]:
mse = function([y_true, y_pred], result)

INFO (theano.gof.compilelock): Waiting for existing lock by process '7650' (I am process '2883')
INFO:theano.gof.compilelock:Waiting for existing lock by process '7650' (I am process '2883')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/novikov/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.11-64/lock_dir
INFO:theano.gof.compilelock:To manually release the lock, delete /home/novikov/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.11-64/lock_dir


In [ ]:
mse([1,2,3,4,5], [1,2,3,4,6])